## Evaluate test set performance
用麦克录制语音输入，通过模型进行预测结果，采用实时语音

In [1]:
# 载入模型
import tensorflow as tf

loaded_model = tf.keras.models.load_model('best_model_part3_sigmoid.h5')

In [2]:
loaded_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 99, 161)]         0         
_________________________________________________________________
sequential (Sequential)      (None, 64)                750400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 750,465
Trainable params: 65
Non-trainable params: 750,400
_________________________________________________________________


In [3]:
command = ['unknown','weiqi']

In [4]:
# 用sklearn.preprocessing.LabelEncoder将指令转换为数字
import sklearn as sk
from sklearn import preprocessing
le = sk.preprocessing.LabelEncoder()
le.fit_transform(command)

array([0, 1], dtype=int64)

In [5]:
import os
import numpy as np
import scipy
from scipy import io
from scipy import signal
from scipy.io import wavfile
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [6]:
# 计算时频谱

def log_specgram(audio, sample_rate, window_size=20, step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate /1e3))
    freqs, times, spec = scipy.signal.spectrogram(audio, fs=sample_rate, window='hann', nperseg=nperseg, noverlap=noverlap, detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

In [7]:
# 输入文件的路径返回MFCC

def get_specgram(signal):
    # sample_rate, signal= scipy.io.wavfile.read(file_path)
    # 统一输入音频文件长度，当音频文件大于1秒时切尾，当音频长度小于1秒时补零
    signal_padding = np.zeros((16000,))
    if len(signal) >= 16000:
        signal_padding = signal[:16000]
    else:        
        signal_padding[:len(signal)] = signal
    # 获取频谱
    _, _, specgram = log_specgram(signal_padding, sample_rate=16000)

    return specgram

In [8]:
def roll_data(roll_data,shift,in_data):    
    roll_data = np.roll(roll_data, shift, axis=0)
    roll_data[shift:] = in_data
    return roll_data

In [9]:
import pyaudio
import numpy as np
import pyttsx3

%matplotlib inline

CHUNK = 40 * 16 # 1ms采样16个点
RATE = 16000
DURATION = 60

plotdata = np.zeros(16000)


result_filter = np.zeros(2, dtype=np.int16)

engine = pyttsx3.init()
p = pyaudio.PyAudio()
stream = p.open(format=pyaudio.paInt16, channels=1, rate=RATE, input=True, frames_per_buffer=CHUNK)

# int(RATE / CHUNK * DURATION)
for i in range(0, int(RATE / CHUNK * DURATION)):
    data = stream.read(CHUNK)
    np_data = np.frombuffer(data, dtype=np.int16)
    shift = len(np_data)
    plotdata = roll_data(plotdata, -shift, np_data)

    
    result = loaded_model.predict(get_specgram(plotdata).reshape(-1,99,161))
    
    result_bool = result.reshape(1) > 0.6
    result_filter = roll_data(result_filter, -1, result_bool)

    
    if (result_filter[-1] ==1) and (result_filter[-2] ==0):
        print('weiqi')
        engine.say("围棋")
        engine.runAndWait()
    
stream.stop_stream()
stream.close()
p.terminate()
engine.stop()

KeyboardInterrupt: 